<a href="https://colab.research.google.com/github/nebulousman/labeling_reviews/blob/main/read_labeled_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Read Excel files with Labeled Reviews into DataFrame**
Dariush Moghadam

An earlier notebook (https://colab.research.google.com/drive/1xIfchmeOSgsQkkMRIRnWJiG6JSDlQs_e?authuser=1#scrollTo=oz9FSt-myoVj) selected random collection of consumer reviews of herbal products and distributed to volunteers for labeling.

This code reads the returned excel spreadsheets with labels and appends the Master Data containing returned/labeled reviews. Filename 'returned_labels_MASTER.csv' will then be processed and cleaned into a separate file for future analysis & addition to the original reviews file. As more labeled data is added to 'returned_labels_MASTER.csv' the cleaning process can be repeated.

This code also archives the labeled xlsx files that have been processed in a separate folder named 'processed'.

In [ ]:
#connect to google drive for dataset
# mount google drive for access to dataset previously added to the drive
from google.colab import drive
drive.mount('/content/drive')
#if you need to remount drive
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#imports
import pandas as pd
import openpyxl
from pathlib import Path
import shutil
import os

In [ ]:
path = '/content/drive/My Drive/Analysis/labeling_reviews'

df = pd.concat([pd.read_excel(f, engine='openpyxl').dropna(how='all').dropna(axis=1,how='all')
                for f in Path(path+'/labeled_reviews').rglob('*.xlsx')])

In [ ]:
df.shape

In [ ]:
#open previously imported xlsx files and append new data
df_previous = pd.read_csv(path +'/master_files/returned_labels_MASTER.csv')

#concatenate dataframes - stack on top of each other
df = pd.concat([df_previous,df], axis=0)

In [ ]:
#save file - note this overwrites previous version of returned_labels_MASTER.csv 
df.to_csv(path +'/master_files/returned_labels_MASTER.csv', index=False)

In [ ]:
#move processed files to new directory called 'processed'
source = path + '/labeled_reviews'
destination = path+'/master_files/labeled_reviews_originals'

file_names = [f for f in os.listdir(source) if f[-4:] == 'xlsx']
    
for file_name in file_names:
    shutil.move(os.path.join(source, file_name), destination)